# Step1. Setup paths

In [1]:
import os
from functions.setup_aws import *

os.chdir('/home/mic/Desktop/Udacity 2020/data engineering/Projects Repo/data_engineering_udacity/Proj2 AWS DWH/')

# Step2. Setup AWS

### Step 2a. Setup Redshift cluster

In [ ]:
#create a client for Redshift
redshift_admin = create_redshift_admin('configs/iam.cfg')

In [ ]:
#create a Redshift cluster with specs defined in aws.cfg
response = create_cluster('configs/aws.cfg', redshift_admin)

In [ ]:
# update aws.cfg with some key cluster parameters

# get cluster parameters
cluster_id = response['Cluster']['ClusterIdentifier']
cluster_params = redshift_admin.describe_clusters(ClusterIdentifier = cluster_id)['Clusters'][0]

cl_vpc_id = cluster_params['VpcId']
cl_vpc_sg_id = cluster_params['VpcSecurityGroups'][0]['VpcSecurityGroupId']
cl_endpoint_adress = cluster_params['Endpoint']['Address']
cl_endpoint_port = cluster_params['Endpoint']['Port']

# update aws.cfg
config = configparser.ConfigParser()
config.read_file(open('configs/aws.cfg'))

config['CLUSTER']['cl_vpc_id'] = cl_vpc_id
config['CLUSTER']['cl_vpc_sg_id'] = cl_vpc_sg_id
config['DB']['db_host'] = cl_endpoint_adress
config['DB']['db_port'] = str(cl_endpoint_port)

with open('configs/aws.cfg', 'w') as configfile:
    config.write(configfile)

configfile.close()    

### Step2b. Open endpoint for inbound connections

In [ ]:
#create ec2 instance
ec2 = create_ec2_instance('configs/iam.cfg')

In [ ]:
#open an incoming TCP port to access the cluster ednpoint
open_tcp_endpoint(ec2, 'configs/aws.cfg')

#### Test connection

In [ ]:
#test connection
conn = make_connection('configs/aws.cfg')
print(conn)
conn.close()

# Step3. Create tables

In [2]:
%run 'functions/create_tables.py'

# Clean-up

In [ ]:
#delete cluster
delete_cluster(cluster_id, redshift_admin)

In [ ]:
#verify cluster deletion
try:
    status = redshift_admin.describe_clusters(ClusterIdentifier = cluster_id)['Clusters'][0]['ClusterStatus']
    print('Cluster status: {}'.format(status))
except Exception as e:
    print(e)